# SimpleFeatures_kNN.ipynb

Model trained on *simple features* that aims at predicting the *emotion labels*. 

The approach consists on getting the arrays of simple features (in *preprocessing*) and train a *k nearest neigbhors* with *sklearn*.
The problem is treated as a *regression* problem, where the loss function can be chosen between *mean squared error*, *-cosine similarity*, or *both*, being a combination of both losses with equal weight.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [2]:
# Data import
genres = ["classical", "electronic", "pop", "rock"]

ZC = np.load('../preprocessing/zeroCrossings_frame200ms_hop100ms.npy')
SC = np.load('../preprocessing/spectralCentroid_frame200ms_hop100ms.npy')
SV = np.load('../preprocessing/spectralVariance_frame200ms_hop100ms.npy')

ZC = (ZC - np.nanmean(ZC)) / np.nanstd(ZC)
SC = (SC - np.nanmean(SC)) / np.nanstd(SC)
SV = (SV - np.nanmean(SV)) / np.nanstd(SV)


features = np.stack((ZC, SC, SV), axis=1)
print(np.shape(features))

(400, 3, 598)


In [3]:
label_songs = pd.read_csv("../preprocessing/labels.csv")
info_songs = pd.read_csv("../Info/info.csv")

select_label = "genre"

if select_label == "genre":
    labels = label_songs[select_label].map({"classical":0, "electronic":1, "pop":2, "rock":3})
labels = labels.to_numpy()

In [4]:
n_data = len(label_songs)
idx = np.random.permutation(n_data)
info_songs = info_songs.reindex(idx)
features = features[idx, :, :]
labels = labels[idx]

In [5]:
fraction_validation = 0.25
n_test = int(fraction_validation*len(labels))
n_train = len(labels) - n_test

train_features, train_labels = features[:n_train], labels[:n_train]
validation_features, validation_labels = features[n_train:], labels[n_train:]

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold

n_neighbors = [1, 5, 10, 15, 20]

k = 5


accuracies = {}

for n in n_neighbors:
        
    knn = KNeighborsClassifier(n_neighbors = n)

    kfold_train_metrics = []
    kfold_test_metrics = []

    total_input = features
    total_labels = labels

    n_data = len(total_input)
    
    cv = KFold(n_splits=k)
    
    accuracy = []

    for train_index, test_index in cv.split(total_input):

        train_df, train_labels = total_input[train_index][:, :, 10], total_labels[train_index]
        test_df, test_labels = total_input[test_index][:, :, 10], total_labels[test_index]

        mean = train_df.mean()
        std = train_df.std()

        train_df = (train_df - mean) / std
        test_df = (test_df - mean) / std

        knn.fit(train_df, train_labels)
        prediction = knn.predict(test_df)

        cm = confusion_matrix(test_labels, prediction)
        
        accuracy.append(np.trace(cm)/np.sum(cm))
    
    accuracies[n] = accuracy

In [7]:
for n in n_neighbors:
    acc_list = accuracies[n]
    
    acc_mean = str(100*np.mean(acc_list))[:5]
    acc_std = str(100*np.std(acc_list))[:3]
    
    accuracies[n] = f"{acc_mean} ± {acc_std}"

In [8]:
print(accuracies)

{1: '33.25 ± 5.0', 5: '35.25 ± 2.6', 10: '35.25 ± 3.9', 15: '35.75 ± 3.4', 20: '34.5 ± 2.1'}
